In [17]:
# library for data analsysis
import pandas as pd
# library to handle data in a vectorized manner
import numpy as np
# library to handle JSON files
import json
import msgpack
!conda install -c conda-forge geopy --yes
# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim
# library to handle requests
import requests
# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
!pip install folium
# map rendering library
import folium
!conda install -c conda-forge geocoder -y
print('Imported Libraries successfully!')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  21.92 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  34.65 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  56.44 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  35.21 MB/s
vincent-0.4.4- 100% |###################

Get the data from url and save as csv using pandas

In [88]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

table = pd.read_html(url, header=0,keep_default_na=False) 

df = table[0]

df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Rename Postcode to PostalCode and Neighbourhood to Neighborhood

In [89]:
df.columns = ['PostalCode','Borough','Neighborhood']
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


process cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [90]:
df1 = df.query('Borough != "Not assigned"').reset_index(drop=True)
df1.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


More than one neighborhood in the same postalcode and combine the neighborhood separated by commas

In [91]:
df2=df1.groupby('PostalCode', as_index=False).agg(lambda x: ', '.join(set(x.dropna())))
df2.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [14]:
df2.loc[df2['Neighborhood'] == 'Not assigned', 'Neighborhood' ] = df2['Borough']
df2.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Use the .shape method to print the number of rows of your dataframe

In [92]:
df2.shape[0]

103

# Segmenting and Clustering Neighborhoods in Toronto - Assignment 2

How to get geographical coordinates of each neighborhood.

In [21]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 


import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.18.1                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge


In [93]:
url="http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv"
coordinates=pd.read_csv(url)
coordinates.columns = ['PostalCode', 'Latitude', 'Longitude']
df1 = pd.merge(df,coordinates, on="PostalCode")

df1 = df1[df1['Borough'].str.contains('Toronto')].reset_index(drop=True)
df1.head(5)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418


Use geopy library to get the latitude and longitude values of Toronto

In [127]:
address = 'Toronto, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Toronto City are 43.653963, -79.387207.


Access to FourSquare

In [96]:
CLIENT_ID = 'YTELCJ2GW1SU1MT2HASDHGX5OX5YUBDYCIHDAD5BUPEK3OCT' # your Foursquare ID
CLIENT_SECRET = 'QJJNNC3HHZSPDDAEVS0S2D20NIU1SR1BPWUHGD0UZDC2Y2PR' # your Foursquare Secret
VERSION = '20181130' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YTELCJ2GW1SU1MT2HASDHGX5OX5YUBDYCIHDAD5BUPEK3OCT
CLIENT_SECRET:QJJNNC3HHZSPDDAEVS0S2D20NIU1SR1BPWUHGD0UZDC2Y2PR


create map of Toronto using latitude and longitude values

In [97]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df1['Latitude'], df1['Longitude'], df1['Borough'], df1['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [130]:
df1.loc[0, 'Neighborhood']

'Harbourfront'

In [131]:
neighborhood_latitude = df1.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df1.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df1.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Harbourfront are 43.6542599, -79.3606359.


In [132]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=YTELCJ2GW1SU1MT2HASDHGX5OX5YUBDYCIHDAD5BUPEK3OCT&client_secret=QJJNNC3HHZSPDDAEVS0S2D20NIU1SR1BPWUHGD0UZDC2Y2PR&v=20181130&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [133]:
results = requests.get(url).json()

In [134]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [135]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

In [136]:
nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Toronto Cooper Koo Family Cherry St YMCA Centre,Gym / Fitness Center,43.653191,-79.357947
3,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
4,Body Blitz Spa East,Spa,43.654735,-79.359874


In [137]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [139]:
toronto_venues = getNearbyVenues(names=df1['Neighborhood'],
                                   latitudes=df1['Latitude'],
                                   longitudes=df1['Longitude']
                                  )

Harbourfront
Regent Park
Ryerson
Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
The Danforth West
Riverdale
Design Exchange
Toronto Dominion Centre
Brockton
Exhibition Place
Parkdale Village
The Beaches West
India Bazaar
Commerce Court
Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North
Forest Hill West
High Park
The Junction South
North Toronto West
The Annex
North Midtown
Yorkville
Parkdale
Roncesvalles
Davisville
Harbord
University of Toronto
Runnymede
Swansea
Moore Park
Summerhill East
Chinatown
Grange Park
Kensington Market
Deer Park
Forest Hill SE
Rathnelly
South Hill
Summerhill West
CN Tower
Bathurst Quay
Island airport
Harbourfront West
King and Spadina
Railway Lands
South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown
St. James Town
First Canadian Place
Underground city


In [140]:
print(toronto_venues.shape)
toronto_venues.head()

(3273, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [141]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Bathurst Quay,13,13,13,13,13,13
Berczy Park,55,55,55,55,55,55
Brockton,18,18,18,18,18,18
Business reply mail Processing Centre969 Eastern,16,16,16,16,16,16
CN Tower,13,13,13,13,13,13
Cabbagetown,46,46,46,46,46,46
Central Bay Street,80,80,80,80,80,80
Chinatown,98,98,98,98,98,98


In [142]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 239 uniques categories.


In [143]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [144]:
toronto_onehot.shape

(3273, 239)

In [145]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Adelaide,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.010000,0.000000,0.0000,0.000000,0.00,0.010000,0.000000,0.000000,0.010000
1,Bathurst Quay,0.000000,0.000000,0.000000,0.076923,0.076923,0.076923,0.153846,0.153846,0.153846,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
2,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
3,Brockton,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
4,Business reply mail Processing Centre969 Eastern,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
5,CN Tower,0.000000,0.000000,0.000000,0.076923,0.076923,0.076923,0.153846,0.153846,0.153846,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
6,Cabbagetown,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
7,Central Bay Street,0.012500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.012500,0.000000,0.0125,0.000000,0.00,0.012500,0.000000,0.000000,0.000000
8,Chinatown,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.051020,0.000000,0.0000,0.051020,0.00,0.010204,0.000000,0.000000,0.000000
9,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000


In [146]:
toronto_grouped.shape

(73, 239)

In [147]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [148]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Steakhouse,American Restaurant,Thai Restaurant,Cosmetics Shop,Clothing Store,Restaurant,Bar,Bakery
1,Bathurst Quay,Airport Lounge,Airport Terminal,Airport Service,Plane,Sculpture Garden,Boat or Ferry,Harbor / Marina,Airport Gate,Airport,Airport Food Court
2,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Seafood Restaurant,Italian Restaurant,Pub,Café,Cheese Shop,Farmers Market,Steakhouse
3,Brockton,Breakfast Spot,Coffee Shop,Café,Pet Store,Furniture / Home Store,Gym,Grocery Store,Convenience Store,Caribbean Restaurant,Performing Arts Venue
4,Business reply mail Processing Centre969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Comic Shop,Restaurant,Burrito Place
5,CN Tower,Airport Lounge,Airport Terminal,Airport Service,Plane,Sculpture Garden,Boat or Ferry,Harbor / Marina,Airport Gate,Airport,Airport Food Court
6,Cabbagetown,Coffee Shop,Restaurant,Bakery,Indian Restaurant,Italian Restaurant,Café,Pub,Market,Pizza Place,Pharmacy
7,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Chinese Restaurant,Bar,Sandwich Place,Ice Cream Shop,Burger Joint,Thai Restaurant,Bubble Tea Shop
8,Chinatown,Café,Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Bakery,Mexican Restaurant,Chinese Restaurant,Comfort Food Restaurant,Dumpling Restaurant
9,Christie,Café,Grocery Store,Park,Diner,Italian Restaurant,Baby Store,Restaurant,Nightclub,Convenience Store,Coffee Shop


In [149]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_[0:10] 
kmeans.labels_

array([0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 2, 0, 0, 3, 0, 0, 0, 0, 2, 1, 0, 0,
       0, 0, 3, 4, 0, 0, 1, 2, 0, 0, 0, 3, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0], dtype=int32)

In [150]:
df1

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,"KMeans(algorithm='auto', copy_x=True, init='k-..."
1,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,"KMeans(algorithm='auto', copy_x=True, init='k-..."
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937,"KMeans(algorithm='auto', copy_x=True, init='k-..."
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937,"KMeans(algorithm='auto', copy_x=True, init='k-..."
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,"KMeans(algorithm='auto', copy_x=True, init='k-..."
5,M4E,East Toronto,The Beaches,43.676357,-79.293031,"KMeans(algorithm='auto', copy_x=True, init='k-..."
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,"KMeans(algorithm='auto', copy_x=True, init='k-..."
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,"KMeans(algorithm='auto', copy_x=True, init='k-..."
8,M6G,Downtown Toronto,Christie,43.669542,-79.422564,"KMeans(algorithm='auto', copy_x=True, init='k-..."
9,M5H,Downtown Toronto,Adelaide,43.650571,-79.384568,"KMeans(algorithm='auto', copy_x=True, init='k-..."


In [152]:
#['Neighborhood','Latitude','Longitude']
result = toronto_grouped.join(df1.set_index('Neighborhood'), on='Neighborhood')
#result = pd.concat([toronto_grouped,df_codes_join],axis=1, join='inner', on='Neighborhood')
result

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,PostalCode,Borough,Latitude,Longitude,Cluster Labels
0,Adelaide,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.010000,0.000000,0.000000,0.010000,M5H,Downtown Toronto,43.650571,-79.384568,"KMeans(algorithm='auto', copy_x=True, init='k-..."
1,Bathurst Quay,0.000000,0.000000,0.000000,0.076923,0.076923,0.076923,0.153846,0.153846,0.153846,...,0.00,0.000000,0.000000,0.000000,0.000000,M5V,Downtown Toronto,43.628947,-79.394420,"KMeans(algorithm='auto', copy_x=True, init='k-..."
2,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,M5E,Downtown Toronto,43.644771,-79.373306,"KMeans(algorithm='auto', copy_x=True, init='k-..."
3,Brockton,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,M6K,West Toronto,43.636847,-79.428191,"KMeans(algorithm='auto', copy_x=True, init='k-..."
4,Business reply mail Processing Centre969 Eastern,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,M7Y,East Toronto,43.662744,-79.321558,"KMeans(algorithm='auto', copy_x=True, init='k-..."
5,CN Tower,0.000000,0.000000,0.000000,0.076923,0.076923,0.076923,0.153846,0.153846,0.153846,...,0.00,0.000000,0.000000,0.000000,0.000000,M5V,Downtown Toronto,43.628947,-79.394420,"KMeans(algorithm='auto', copy_x=True, init='k-..."
6,Cabbagetown,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,M4X,Downtown Toronto,43.667967,-79.367675,"KMeans(algorithm='auto', copy_x=True, init='k-..."
7,Central Bay Street,0.012500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.012500,0.000000,0.000000,0.000000,M5G,Downtown Toronto,43.657952,-79.387383,"KMeans(algorithm='auto', copy_x=True, init='k-..."
8,Chinatown,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.010204,0.000000,0.000000,0.000000,M5T,Downtown Toronto,43.653206,-79.400049,"KMeans(algorithm='auto', copy_x=True, init='k-..."
9,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,M6G,Downtown Toronto,43.669542,-79.422564,"KMeans(algorithm='auto', copy_x=True, init='k-..."


In [153]:
toronto_merged = result

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!
toronto_merged.columns.values

ValueError: Length of values does not match length of index

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
## Cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[0] + list(range(toronto_merged.shape[1]-10, toronto_merged.shape[1]))]]

In [ ]:
## Cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[0] + list(range(toronto_merged.shape[1]-10, toronto_merged.shape[1]))]]

In [ ]:
### Cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[0] + list(range(toronto_merged.shape[1]-10, toronto_merged.shape[1]))]]

In [ ]:
#### Cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[0] + list(range(toronto_merged.shape[1]-10, toronto_merged.shape[1]))]]

In [ ]:
#### Cluster 5
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[0] + list(range(toronto_merged.shape[1]-10, toronto_merged.shape[1]))]]